# Deploy Retrieval Augmented Generation Function in Watson Machine Learning

This optional notebook tests the deployed RAG python function from `2-deploy-rag-function-in-wml`. 

**Prerequisites:**
1. Succesfully ran `1-file-ingestion-from-cos` and `2-deploy-rag-function-in-wml` notebooks


**Before running any cells, connect the notebook to the rest of the project by inserting a project token.**

You can do this by clicking the three vertical dots next to the Code Snippet icon on the top right of the notebook UI and then clicking Insert project token to insert the token at the beginning of your notebook. Make sure to execute the new cell at the top of the notebook before running any other notebook cells.

## Configure Notebook
The cells below configure the notebook by importing the necessary packages, connecting to the Watson Machine Learning client, and adding the values configured in your project's Parameter Sets to the namespace.

In [ ]:
import json

import ibm_watson_machine_learning

wslib.download_file("utils.py")
import utils

In [ ]:
wslib_extension = utils.WSLibExtension(wslib)
parameters = wslib_extension.get_all_parameter_set_values()
SPACE_ID = parameters["wml_space_id"]
IBM_CLOUD_APIKEY = parameters["ibm_cloud_apikey"]
IBM_CLOUD_REGION = parameters["ibm_cloud_region"]
WML_DEPLOYMENT_ID = json.loads(
    wslib.load_data("configured-watsonx-assistant-extension-openapi.json").read()
)["paths"]["/ml/v4/deployments/{wml_deployment_id}/predictions"]["post"]["parameters"][0]["schema"]["default"]

In [ ]:
wml_credentials = {
    "url": f"https://{IBM_CLOUD_REGION}.ml.cloud.ibm.com",
    "apikey": IBM_CLOUD_APIKEY,
}
wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)
wml_client.set.default_space(SPACE_ID)

## Test Endpoint
The cell below tests the deployed Python function by calling the deployment against a set of user queries and reformatting the deployment output to a cleaner format.

In [ ]:
questions = [
    "What are some key features of Watsonx.data?",
    "What are some key features of Watsonx.governance?",
    "What are some key features of Watsonx.ai?",
    "How does Watsonx.ai work with Watsonx.governance?",
    "When was Watsonx.data GAed?",
]

for question in questions:
    wx_result = wml_client.deployments.score(
        WML_DEPLOYMENT_ID, utils.format_scoring_input(question)
    )
    utils.display_wml_response(question, wx_result)